## Preprocessing

In [70]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Optimization No. 3
1. using optimized model 2, add NAME and create buckets
2. add more APPLICATION_TYPE and CLASSIFICATION buckets

In [71]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN'], axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [72]:
# Look at NAME value counts to identify and replace with "Other"
application_df["NAME"].value_counts()[application_df["NAME"].value_counts() > 100]

,count
NAME,
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,408
PTA TEXAS CONGRESS,368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,331
ALPHA PHI SIGMA,313
TOASTMASTERS INTERNATIONAL,293


In [73]:
# Choose a cutoff value and create a list of name types to be replaced
# use the variable name `name_types_to_replace`
name_types_to_replace = application_df["NAME"].value_counts()[application_df["NAME"].value_counts() < 100].index.tolist()

# Replace in dataframe
application_df["NAME"] = application_df["NAME"].replace(name_types_to_replace, "Other")

# Check to make sure replacement was successful
application_df["NAME"].value_counts()

,count
NAME,
Other,25987
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,408
PTA TEXAS CONGRESS,368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,331
ALPHA PHI SIGMA,313


In [74]:
# Look at INCOME_AMT value counts to identify and replace with "Other"
application_df["INCOME_AMT"].value_counts()

,count
INCOME_AMT,
0,24388
25000-99999,3747
100000-499999,3374
1M-5M,955
1-9999,728
10000-24999,543
10M-50M,240
5M-10M,185
50M+,139


In [75]:
import numpy as np

# Define the conditions
conditions = [
    application_df['INCOME_AMT'] == "0",
    application_df['INCOME_AMT'] == "1-9999",
    application_df['INCOME_AMT'] == "10000-24999",
    application_df['INCOME_AMT'] == "25000-99999",
    application_df['INCOME_AMT'] == "100000-499999",
    application_df['INCOME_AMT'] == "1M-5M",
    application_df['INCOME_AMT'] == "5M-10M",
    application_df['INCOME_AMT'] == "10M-50M"
]

# Define the corresponding values for LOW_INCOME_AMT and HIGH_INCOME_AMT
low_income_values = [0, 1, 10000, 25000, 100000, 1000000, 5000000, 10000000]
high_income_values = [0, 9999, 24999, 99999, 499999, 4999999, 9999999, 49999999]

# Use np.select to assign values based on conditions
application_df['LOW_INCOME_AMT'] = np.select(conditions, low_income_values, default=50000000)
application_df['HIGH_INCOME_AMT'] = np.select(conditions, high_income_values, default=50000000)

# Drop the INCOME_AMT column
application_df = application_df.drop("INCOME_AMT", axis=1)

application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOW_INCOME_AMT,HIGH_INCOME_AMT
0,Other,T10,Independent,C1000,ProductDev,Association,1,N,5000,1,0,0
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1,1,9999
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0,0,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1,10000,24999
4,Other,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1,100000,499999


In [76]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
NAME,31
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [77]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [78]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_df["APPLICATION_TYPE"].value_counts() < 60].index.tolist()

# Replace in dataframe
application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(application_types_to_replace, "Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [79]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [80]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [81]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < 50].index.tolist()

# Replace in dataframe
application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(classifications_to_replace, "Other")

# Check to make sure replacement was successful
application_df["CLASSIFICATION"].value_counts()


,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
Other,391
C1700,287
C4000,194


In [82]:
application_df.head(15)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOW_INCOME_AMT,HIGH_INCOME_AMT
0,Other,T10,Independent,C1000,ProductDev,Association,1,N,5000,1,0,0
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1,1,9999
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0,0,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1,10000,24999
4,Other,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1,100000,499999
5,Other,T3,Independent,C1200,Preservation,Trust,1,N,5000,1,0,0
6,Other,T3,Independent,C1000,Preservation,Trust,1,N,31452,1,100000,499999
7,Other,T3,Independent,C2000,Preservation,Trust,1,N,7508025,1,10000000,49999999
8,Other,T7,Independent,C1000,ProductDev,Trust,1,N,94389,1,1,9999
9,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0,0,0


In [83]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [84]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df).astype(int)
application_df.head(15)

,STATUS,ASK_AMT,IS_SUCCESSFUL,LOW_INCOME_AMT,HIGH_INCOME_AMT,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,1,9999,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,10000,24999,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,100000,499999,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
5,1,5000,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
6,1,31452,1,100000,499999,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
7,1,7508025,1,10000000,49999999,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
8,1,94389,1,1,9999,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
9,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [85]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop("IS_SUCCESSFUL", axis=1).values
y = application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [86]:
# Create a StandardScaler instances
X_scaler = StandardScaler()

# Fit the StandardScaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Count the columns in a DataFrame X
features_count = len(X_train[0])
neurons_first_layer = features_count + 24
neurons_second_layer = features_count
neurons_thrird_layer = features_count

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=neurons_first_layer, activation="relu", input_dim=features_count))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=neurons_second_layer, activation="relu"))

# Thrird hidden layer
nn_model.add(tf.keras.layers.Dense(units=neurons_thrird_layer, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 103)                 │           8,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 79)                  │           8,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 79)                  │           6,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              80 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,856 (89.28 KB)

 Trainable params: 22,856 (89.28 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7312 - loss: 0.5295
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7560 - loss: 0.4948
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7587 - loss: 0.4882
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7573 - loss: 0.4922
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7553 - loss: 0.4891
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7565 - loss: 0.4891
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7583 - loss: 0.4858
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7640 - loss: 0.4860
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7568 - loss: 0.4895
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7615 - loss: 0.4865
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7627 - loss: 0.4849
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7539 - loss: 0.5267
Loss: 0.5267384052276611, Accuracy: 0.7539358735084534


In [91]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization3.h5')